In [1]:
from pathlib import Path

# Base paths (notebook is in moviechat/notebooks)
ROOT = Path("..").resolve()
SRC = ROOT / "src"
CORE = SRC / "core"
API = SRC / "api"

# 1) Ensure folders exist
CORE.mkdir(parents=True, exist_ok=True)
API.mkdir(parents=True, exist_ok=True)

# 2) Write .env with your keys
env_text = """TMDB_API_KEY=b5407560fba93b51806a475ec7d72803
WATCHMODE_API_KEY=NiqnjBWAn1tpsYYqkvDoWlfLizD5J2iyumUPSRD0
DEFAULT_REGION=US
"""
(ROOT / ".env").write_text(env_text, encoding="utf-8")

# 3) init files
(SRC / "__init__.py").write_text("", encoding="utf-8")
(CORE / "__init__.py").write_text("", encoding="utf-8")
(API / "__init__.py").write_text("", encoding="utf-8")

# 4) providers.py
providers_code = """import os
from pathlib import Path
from typing import Any, Dict, List, Optional

import requests
from dotenv import load_dotenv

# Load .env from project root (moviechat/.env)
load_dotenv(Path(__file__).resolve().parents[2] / ".env")

TMDB_BASE = "https://api.themoviedb.org/3"
WATCHMODE_BASE = "https://api.watchmode.com/v1"


def _tmdb_key() -> str:
    key = os.getenv("TMDB_API_KEY")
    if not key:
        raise RuntimeError("TMDB_API_KEY not found. Put it in moviechat/.env")
    return key


def _watchmode_key() -> str:
    key = os.getenv("WATCHMODE_API_KEY")
    if not key:
        raise RuntimeError("WATCHMODE_API_KEY not found. Put it in moviechat/.env")
    return key


def tmdb_discover_tv(genres: Optional[List[int]] = None, page: int = 1) -> Dict[str, Any]:
    params: Dict[str, Any] = {
        "api_key": _tmdb_key(),
        "page": page,
        "sort_by": "popularity.desc",
        "include_adult": "false",
    }
    if genres:
        params["with_genres"] = ",".join(map(str, genres))

    r = requests.get(f"{TMDB_BASE}/discover/tv", params=params, timeout=20)
    r.raise_for_status()
    return r.json()


def tmdb_discover_movie(genres: Optional[List[int]] = None, page: int = 1) -> Dict[str, Any]:
    params: Dict[str, Any] = {
        "api_key": _tmdb_key(),
        "page": page,
        "sort_by": "popularity.desc",
        "include_adult": "false",
    }
    if genres:
        params["with_genres"] = ",".join(map(str, genres))

    r = requests.get(f"{TMDB_BASE}/discover/movie", params=params, timeout=20)
    r.raise_for_status()
    return r.json()


def tmdb_upcoming_movies(page: int = 1) -> Dict[str, Any]:
    params = {"api_key": _tmdb_key(), "page": page}
    r = requests.get(f"{TMDB_BASE}/movie/upcoming", params=params, timeout=20)
    r.raise_for_status()
    return r.json()


def watchmode_search(title: str) -> Dict[str, Any]:
    params = {
        "apiKey": _watchmode_key(),
        "search_field": "name",
        "search_value": title,
    }
    r = requests.get(f"{WATCHMODE_BASE}/search/", params=params, timeout=20)
    r.raise_for_status()
    return r.json()


def watchmode_sources(title_id: int, region: str = "US") -> List[Dict[str, Any]]:
    params = {"apiKey": _watchmode_key(), "regions": region}
    r = requests.get(f"{WATCHMODE_BASE}/title/{title_id}/sources/", params=params, timeout=20)
    r.raise_for_status()
    return r.json()
"""
(CORE / "providers.py").write_text(providers_code, encoding="utf-8")

# 5) recommender.py
recommender_code = """from typing import Dict, List, Optional, Tuple

from .providers import (
    tmdb_discover_movie,
    tmdb_discover_tv,
    tmdb_upcoming_movies,
    watchmode_search,
    watchmode_sources,
)

TMDB_GENRES = {
    "action": 28,
    "comedy": 35,
    "crime": 80,
    "thriller": 53,
    "animation": 16,
    "drama": 18,
    "mystery": 9648,
    "romance": 10749,
    "horror": 27,
    "sci-fi": 878,
    "fantasy": 14,
}

SUBSCRIPTION_ALIASES = {
    "netflix": "Netflix",
    "prime video": "Amazon Prime",
    "amazon prime": "Amazon Prime",
    "hulu": "Hulu",
    "disney+": "Disney+",
    "max": "HBO Max",
    "hbo max": "HBO Max",
    "apple tv+": "AppleTV+",
    "crunchyroll": "Crunchyroll",
    "paramount+": "Paramount+",
    "peacock": "Peacock",
}


def normalize_subs(subs: List[str]) -> List[str]:
    out = []
    for s in subs:
        key = s.strip().lower()
        out.append(SUBSCRIPTION_ALIASES.get(key, s.strip()))
    return list(dict.fromkeys(out))


def extract_genre_ids(text: str) -> List[int]:
    t = text.lower()
    ids: List[int] = []
    for name, gid in TMDB_GENRES.items():
        if name in t:
            ids.append(gid)
    if "thrill" in t and TMDB_GENRES["thriller"] not in ids:
        ids.append(TMDB_GENRES["thriller"])
    return ids


def is_anime(item: Dict) -> bool:
    # Practical heuristic for anime
    return item.get("original_language") == "ja"


def _best_watchmode_match(title: str) -> Optional[int]:
    data = watchmode_search(title)
    results = data.get("title_results", [])
    if not results:
        return None
    return results[0].get("id")


def _filter_by_subscriptions(title: str, subs: List[str], region: str) -> Tuple[bool, List[Dict]]:
    if not subs:
        return True, []

    wm_id = _best_watchmode_match(title)
    if wm_id is None:
        return False, []

    sources = watchmode_sources(wm_id, region=region)
    subs_norm = set(normalize_subs(subs))

    matched = []
    for s in sources:
        name = s.get("name") or s.get("source") or ""
        if name in subs_norm:
            matched.append(s)

    return (len(matched) > 0), matched


def recommend(
    content_type: str,
    prompt: str,
    subscriptions: Optional[List[str]] = None,
    region: str = "US",
    include_international: bool = True,
    limit: int = 10,
) -> List[Dict]:
    content_type = (content_type or "").lower().strip()
    subs = subscriptions or []

    genre_ids = extract_genre_ids(prompt)

    if content_type == "anime" and 16 not in genre_ids:
        genre_ids = genre_ids + [16]

    candidates: List[Dict] = []
    for page in (1, 2):
        if content_type == "movie":
            data = tmdb_discover_movie(genres=genre_ids, page=page)
            candidates += data.get("results", [])
        else:
            data = tmdb_discover_tv(genres=genre_ids, page=page)
            candidates += data.get("results", [])

    if content_type == "anime":
        candidates = [c for c in candidates if is_anime(c)]

    if not include_international:
        candidates = [c for c in candidates if c.get("original_language") in ("en", None)]

    out: List[Dict] = []
    seen = set()

    for item in candidates:
        if len(out) >= limit:
            break

        if content_type == "movie":
            title = item.get("title")
            tmdb_id = item.get("id")
            if not title or tmdb_id in seen:
                continue
            seen.add(tmdb_id)

            ok, matched_sources = _filter_by_subscriptions(title, subs, region)
            if not ok:
                continue

            out.append({
                "type": "movie",
                "title": title,
                "overview": item.get("overview"),
                "rating": item.get("vote_average"),
                "release_date": item.get("release_date"),
                "language": item.get("original_language"),
                "tmdb_id": tmdb_id,
                "where_to_watch": matched_sources,
            })

        else:
            title = item.get("name")
            tmdb_id = item.get("id")
            if not title or tmdb_id in seen:
                continue
            seen.add(tmdb_id)

            ok, matched_sources = _filter_by_subscriptions(title, subs, region)
            if not ok:
                continue

            out.append({
                "type": "anime" if content_type == "anime" else "series",
                "title": title,
                "overview": item.get("overview"),
                "rating": item.get("vote_average"),
                "first_air_date": item.get("first_air_date"),
                "language": item.get("original_language"),
                "tmdb_id": tmdb_id,
                "where_to_watch": matched_sources,
            })

    return out


def upcoming(limit: int = 10) -> List[Dict]:
    data = tmdb_upcoming_movies(page=1)
    results = data.get("results", [])[:limit]
    return [
        {"type": "upcoming_movie", "title": x.get("title"), "release_date": x.get("release_date"), "tmdb_id": x.get("id")}
        for x in results
    ]
"""
(CORE / "recommender.py").write_text(recommender_code, encoding="utf-8")

# 6) app.py
app_code = """from typing import List

from fastapi import FastAPI
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# Load env FIRST so keys are available for imports
load_dotenv()

from src.core.recommender import recommend, upcoming

app = FastAPI(title="MovieChat API")

class RecommendRequest(BaseModel):
    content_type: str = Field(..., description="movie | series | anime")
    prompt: str = Field(..., description="e.g. crime thriller but also comedy")
    subscriptions: List[str] = Field(default_factory=list, description="e.g. ['Netflix','Hulu']")
    region: str = Field(default="US", description="e.g. US, IN, CA")
    include_international: bool = True
    limit: int = 10

@app.get("/health")
def health():
    return {"ok": True}

@app.post("/recommend")
def post_recommend(req: RecommendRequest):
    return recommend(
        content_type=req.content_type,
        prompt=req.prompt,
        subscriptions=req.subscriptions,
        region=req.region,
        include_international=req.include_international,
        limit=req.limit,
    )

@app.get("/upcoming")
def get_upcoming(limit: int = 10):
    return upcoming(limit)
"""
(API / "app.py").write_text(app_code, encoding="utf-8")

print("✅ DONE: wrote .env + init files + providers.py + recommender.py + app.py")
print("Project root:", ROOT)
print("API file:", (API / "app.py"))


✅ DONE: wrote .env + init files + providers.py + recommender.py + app.py
Project root: C:\Users\ASUS\moviechat
API file: C:\Users\ASUS\moviechat\src\api\app.py
